### Dice trading

##### 4.a)

$S_t = \{p_t, d_t\}$, avec $p_t$ le nombre de points et $d_t$ le nombre de dés

$a_t = \{0, 1\}$, soit acheter un nouveau dé, soit ne rien faire

$p_t \geq 0$

$d_t \leq 3$

$d_{t+1} = d_t + a_t$

$p_{t+1} = p_t - 5*a_t + \max_{i \in \{1, d_t\}} (u_i (1, 2, 3, 4, 5, 6))$, le nombre de points est déterminé par le score max de tous les dés

##### 4.b) Implement an heuristic

In [1]:
function never_buy_heuristic(s,t) # Always returns "Don't Buy" (never buy dice) 
    return "Don't Buy"
end

never_buy_heuristic (generic function with 1 method)

##### 4.c) Write a simulator giving $V_{\pi}$ in a confidence interval of 95%

In [2]:
using Pkg
Pkg.add("Distributions")

   Resolving package versions...
  No Changes to `C:\Users\Ponts\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Ponts\.julia\environments\v1.9\Manifest.toml`


In [15]:
using Distributions
using Random

function simulator_simple_game(heuristic, history=false, T=10)
    nb_dice = 1
    total_points = 0
    println("test")
    if history
        history_list = []
    end

    for turn in 1:T
        action = heuristic((total_points, nb_dice), turn)
        @assert action != "Buy" || total_points >= 6 "RuleError You don't have enough points to buy a dice"
        if action == "Buy" && total_points >= 6 # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
            nb_dice += 1
            total_points -= 5
        end
        dice_roll = maximum(rand(1:6, nb_dice))
        total_points += dice_roll

        if history
            push!(history_list, (action, dice_roll, total_points))
        end
    end
    
    if history
        return total_points, history_list
    end
    return total_points
end
simulator_simple_game(never_buy_heuristic, true, 10)

test


(36, Any[("Don't Buy", 4, 4), ("Don't Buy", 2, 6), ("Don't Buy", 2, 8), ("Don't Buy", 5, 13), ("Don't Buy", 2, 15), ("Don't Buy", 4, 19), ("Don't Buy", 4, 23), ("Don't Buy", 2, 25), ("Don't Buy", 5, 30), ("Don't Buy", 6, 36)])

In [16]:
function simulator_simple_game_V(heuristic, T=10, N= 100)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    nb_dice = 1
    total_points = 0
    Liste = []
    for j in 1:N
        total_points = 0
        for turn in 1:T
            action = heuristic((total_points, nb_dice), turn)
            @assert action != "Buy" || total_points >= 6 "RuleError You don't have enough points to buy a dice"
            if action == "Buy" && total_points >= 6 # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
            dice_roll = maximum(rand(1:6, nb_dice))
            total_points += dice_roll
            
        end
        push!(Liste, total_points)
    end
    return mean(Liste), [mean(Liste) - 1.96*std(Liste)/sqrt(N), mean(Liste) + 1.96*std(Liste)/sqrt(N)]
end
simulator_simple_game_V(never_buy_heuristic, 10)

(34.83, [33.74279544006129, 35.9172045599387])

In [5]:
# function compute_expected_cost(heuristic, p=0.05):
#     #p is the error level
#     n=0 # Number of game simulated
#     emp_mean=0
#     emp_dev=0

#     d = Normal()  # Crée une distribution normale standard
#     p = 0.95  # Probabilité pour laquelle vous voulez trouver le quantile
#     q = quantile(d, p)  # Calcule le quantile
    
#     while n<1 and 
#     end
# end

ErrorException: syntax: whitespace not allowed after ":" used for quoting